# <center> Kafka Consumer for Reddit </center>

In [ ]:
from confluent_kafka import Consumer, KafkaException, KafkaError
import sys
import getopt
import json
from pprint import pformat

In [ ]:
def stats_cb(stats_json_str):
    stats_json = json.loads(stats_json_str)
    print('\nKAFKA Stats: {}\n'.format(pformat(stats_json)))

broker = 'clnode225.clemson.cloudlab.us:6667'
group = '1'
topics = ['test']
    
# Consumer configuration
# See https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md
conf = {'bootstrap.servers': broker, \
        'group.id': group, \
        'session.timeout.ms': 6000,\
        'default.topic.config': {'auto.offset.reset': 'smallest'}}

In [ ]:
# Create Consumer instance
c = Consumer(**conf)

def print_assignment(consumer, partitions):
    print('Assignment:', partitions)

# Subscribe to topics
c.subscribe(topics, on_assign=print_assignment)

# Read messages from Kafka, print to stdout
try:
    while True:
        msg = c.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            # Error or event
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' % (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error():
                # Error
                raise KafkaException(msg.error())
        else:
            # Proper message
            # sys.stderr.write('%% %s [%d] at offset %d with key %s:\n' % (msg.topic(), msg.partition(), msg.offset(), str(msg.key())))
            #jsonData = json.loads(json.loads(msg.value()))
            jsonData = json.loads(msg.value())
            # print (jsonData['text'])
            print(msg.value())

except KeyboardInterrupt:
    sys.stderr.write('%% Aborted by user\n')

# Close down consumer to commit final offsets.
c.close()

### Challenge Activity:

- Instructor will set up an infinitely running Kafka producer to grab Reddit's latest posts.

- Students develop multiple consumers, each performs one of the following tasks:
  - Save all incoming data into a file called **raw.json**.
  - Monitors and keep tracks of number of submissions coming to various subreddits. Every five minutes, save the top-5 subreddits with the most submissions to an external file. 
  - Extract the original URL, the subreddit, and the author of each post coming in from the stream. Store this information in a tab-separated file called **posts.txt**. 
  - After 10-minute, begin examine the original URL of each post in **posts.txt**. Create a new file called **posts_stat.txt** with the following information: the original URL, the subreddit, the author of each post, number of new comments, number of upvotes/downvotes 